ref: https://takemikami.com/2020/10/31/PDFPython.html

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams, LTTextContainer, LTFigure

from pdfminer.pdfpage import PDFPage
from io import StringIO
import re
import io
from pdfminer.high_level import extract_text

from pdfminer.high_level import extract_text, extract_text_to_fp
import re
import glob
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertJapaneseTokenizer, BertModel

import transformers
import torch
import pandas as pd

#  Sentence BERT のコードはトークナイザとして BertTokenize を決め打ちで使用します。Hagging Face で公開されている日本語事前学習済みモデルを利用できるように、以下のように差し替えてしまいます。
transformers.BertTokenizer = transformers.BertJapaneseTokenizer

from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from subprocess import call

In [2]:
re_list = [
    r'(([０-９]|[0-9])[\．|\.]\s*(.+?)\s*[…|\.|·]+\s*[p|P]*[.|．]*\s*([０-９]+|[0-9]+))',
    r'((（[０-９]）|（[0-9]）)\s*(.+?)\s*[…|\.|·]+\s*[p|P]*[.|．]*\s*([０-９]+|[0-9]+))',
    r'((\([０-９]\)|\([0-9]\))\s*(.+?)\s*[…|\.|·]+\s*[p|P]*[.|．]*\s*([０-９]+|[0-9]+))'
]

leader_list = ['……','.....','·····']

In [2]:
class PDFProcessor:
    def __init__(self, input_file:str, leader_list:list, re_list:list):
        self.input_file = input_file
        self.leader_list = leader_list
        self.re_list = re_list
        self.mokuji=[]
        self.mokuji_page_number = None
        
        self.first_index = None
        self.second_index = None
        
        self.extract_area = range(3,20) # Noneだと、全ページを抽出する
#         self.extract_titles = []
    
    #  re_listで定義した正規表現に一致するものを目次データとして抜き出す関数
    def parse_outline(self):
        rsrcmgr = PDFResourceManager()
        mokuji_page = False
        with open(self.input_file, "rb") as fp:
            mokuji_page = False
            for pidx, page in enumerate(PDFPage.get_pages(fp)):
                out_fp = io.StringIO()
                device = TextConverter(
                    rsrcmgr,
                    out_fp,
                    laparams=LAParams(),
                    imagewriter=None
                )
                interpreter = PDFPageInterpreter(rsrcmgr, device)
                interpreter.process_page(page)
                out_fp.seek(0)
                page_str = out_fp.read()
                
                if mokuji_page:
                    break

                for leader in self.leader_list:
                    # Find a page which contains 目次
                    if leader in page_str:
                        print("yes",leader)
                        mokuji_page = True
                        self.mokuji_page_number = pidx
                        for ln in page_str.split('\n'):
                            for re_expression in self.re_list:
                                m = re.match(re_expression, ln)
                                if m != None:
                                    full_title = m.groups()[0]
                                    title_num = m.groups()[1]
                                    title = m.groups()[2]
                                    page_number = m.groups()[-1]
                                else:
                                    continue
                                self.mokuji.append([full_title,title_num,title,page_number])
                                break
                    elif mokuji_page:
                        break
        return self.mokuji,self.mokuji_page_number
    
        
    # 目次の情報をもとに、取り出すページ範囲を決定する
    def define_extract_range(self):
        # 目次ページが不明、または目次内容が読み取れなかった場合は全ページから文章を抜き出す。
        if self.mokuji_page_number == None or self.mokuji==[]:
            print("Mokuji page was not detected. We'll extract whole PDF file. We'll extract p3-20 from PDF file.")
            return self.extract_area
        # 目次情報が取れた場合
        else:
            for i,item in enumerate(self.mokuji):
                if item[1] == "1" or item[1] == "１":
                    self.first_index =  item[-1]

                if item[1] == "2" or item[1] == "２":
                    self.second_index = item[-1]

            # インデックス１とインデックス２どちらかが見つからなかった場合
            if self.first_index == None or self.second_index ==None:
                print("Either Index 1 or Index 2 was not detected.")
                # インデックス１だけ見つかった場合　⇨ インデックス１〜最終ページ
                if self.first_index != None:
                    print("Only Index 1 was detected. We'll extract Index 1 to p20 of PDF file).")
                    self.extract_area = [i for i in range(mokuji_page_number-1+int(self.first_index), 20)]
                    return self.extract_area
                # インデックス2だけ見つかった場合　⇨ 目次ページ+1 ~ インデックス２のページ
                elif self.second_index != None:
                    print("Only Index 2 was detected. We'll extract first page to Index 2 of PDF file.")
                    self.extract_area = [i for i in range(self.mokuji_page_number+1,self.mokuji_page_number+int(self.second_index))]
                    return self.extract_area 
                # インデックス１もインデックス２も見つからなかった場合。pdfの目次ページのレイアウトがおかしい場合は対応不可なので、全ページを対象にする
                else:
                    print("Index was not properly detected from Mokuji page. We'll extract p3-20 from PDF file.")
                    return self.extract_area
                    
            # インデックスが１、２とも見つかった場合
            else:
                #　２nd index pageまで念の為抽出する。
                print("Both Index1 and 2 were detected.")
                self.extract_area = [i for i in range(self.mokuji_page_number-1+int(self.first_index),self.mokuji_page_number+int(self.second_index))]
                return self.extract_area
#                 extract_titles = [re.match("(.+?)(?=…)",j).group() for j in [i[0] for i in mokuji[first_index:second_index]]]

    def extract_text(self):  
        result = extract_text(self.input_file, page_numbers=self.extract_area, codec='utf-8')
#         print(result)
        result_buffer =re.sub("\n","",result)
        result_buffer = re.split("(?<=。(?!）))",result_buffer)
#         result_list = [re.sub("\u3000*\d*(.+?)\s*\((\d{4}\))*\s*(.+?)[期|年度]\s*\u3000*決算短信"," ",i) for i in result_buffer]
        result_list = [re.sub("\u3000*\d*(.+?)\s*\d{4}\s*(.+?)[期|年度][\u3000|\s]*決算短信"," ",i) for i in result_buffer]
        print("ページレンジ：　",self.extract_area[0], " to ", self.extract_area[-1])
        print("総行数：　",len(result_list))
        return result_list

# 動作確認

In [15]:
pp =  PDFProcessor(input_file = "decrypted_PDFs/Olympus_FY2021_Consolidated_Financial_Results_J.pdf", leader_list = ['……','.....','·····'] ,re_list = re_list)
pp.parse_outline()
pp.define_extract_range()
a = pp.extract_text()

print(pp.extract_area)
print(pp.mokuji_page_number)
print(pp.first_index)
print(pp.second_index)
print(a)

yes ……
Either Index 1 or Index 2 was not detected.
Only Index 2 was detected. We'll extract first page to Index 2 of PDF file.
ページレンジ：　 3  to  18
総行数：　 223
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
2
None
17
[' １．経営成績等の概況(1) 経営成績に関する分析（当期の経営成績）業績全般に関する分析 \xa0売\xa0 上\xa0 高営業利益税引前利益2021年３月期 2020年３月期 増 減 率（％）730,544755,231△3.381,98592,200△11.176,81086,617△11.312,91851,670△75.010.05円39.37円－（単位：百万円）親会社の所有者に帰属する当期利益基本的1株当たり 当期利益 \xa0平均為替レート比較表   \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 （単位：円）\xa0対米ドル対ユーロ 対人民元\xa0当期106.06 123.70 15.67 前期108.74 120.82 15.60 \xa0\xa0\xa0\xa0  当連結会計年度における世界経済は、新型コロナウイルス感染症の大流行の影響により、厳しい状況となりました。', '経済活動は段階的に再開し、ワクチン接種も徐々に進んでいるものの、地域によっては感染再拡大の傾向が見られるなど、依然として不確実性の高い状況が続いています。', 'わが国経済においては、輸出において持ち直しの動きがみられ、企業収益への影響も縮小しつつあるものの、新型コロナウイルス感染症の影響により、世界経済と同様に厳しい状況となりました。', '  こうした環境下にあるものの、当社グループは、2019年１月に発表した真のグローバル・メドテックカンパニーへの飛躍を目指した企業変革プラン「Transform Olympus」と、それに基づき2019年11月に発表した中長期の経営戦略に沿って、持続的な成長に向けた取り組みを推し進めています。', '  その中で、当社は「事業ポートフォリオの選択と集中

In [3]:
# transformer = models.BERT('cl-tohoku/bert-base-japanese-whole-word-masking') # class BERT was DEPRECATED: Please use models.Transformer instead.
transformer = models.Transformer('cl-tohoku/bert-base-japanese-whole-word-masking')

# poolingの指定：[CLS]トークンに対応するベクトルを使用する
pooling = models.Pooling(transformer.get_word_embedding_dimension(), pooling_mode_mean_tokens=False, pooling_mode_cls_token=True, pooling_mode_max_tokens=False)

# param modules: This parameter can be used to create custom SentenceTransformer models from scratch.
model = SentenceTransformer(modules=[transformer, pooling])

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# 要約文を辞書化

pdf_directory = "./decrypted_PDFs/*"

input_csv= pd.read_csv('決算短信要約データ解析.csv')

youyakus = input_csv[["決算短信原文","要約"]].values.tolist()

youyaku_sentence_dic = {}
for y in youyakus:
    if y[0] not in youyaku_sentence_dic:
        youyaku_sentence_dic[y[0]] = []
    youyaku_sentence_dic[y[0]].append(y[1])

len(youyaku_sentence_dic.keys())

30

In [6]:
# Threshold = 0.841
def generate_dataset(file, THRESHOLD=0.841):
    pdf_filename = os.path.basename(file)
    
    pp =  PDFProcessor(input_file =file, leader_list = leader_list ,re_list = re_list)
    pp.parse_outline()
    pp.define_extract_range()
    sentence_list = pp.extract_text()

    summary_list = youyaku_sentence_dic[pdf_filename]
#     embed_sentence = model.encode(sentence_list["sentence_list"]).reshape(1,-1)
    labels_list = []
    for i, sentence in enumerate(sentence_list):
        labels_buffer=[]
        embed_sentence = model.encode(sentence).reshape(1,-1)
        for s in summary_list:
            embed_summary_sentence = model.encode(s).reshape(1,-1)
            cos_sim = cosine_similarity(embed_sentence,embed_summary_sentence)[0]
    #             dataset[i] = cos_sim
            if cos_sim >= THRESHOLD:
                labels_buffer.append(1)
            else:
                labels_buffer.append(0)
        if 1 in labels_buffer:
            labels_list.append(1)
        else:
            labels_list.append(0)
    return labels_list, sentence_list

In [7]:
%%time

labels = []
sentences = []

re_list = [
    r'(([０-９]|[0-9])[\．|\.]\s*(.+?)\s*[…|\.|·]+\s*[p|P]*[.|．]*\s*([０-９]+|[0-9]+))',
    r'((（[０-９]）|（[0-9]）)\s*(.+?)\s*[…|\.|·]+\s*[p|P]*[.|．]*\s*([０-９]+|[0-9]+))',
    r'((\([０-９]\)|\([0-9]\))\s*(.+?)\s*[…|\.|·]+\s*[p|P]*[.|．]*\s*([０-９]+|[0-9]+))'
]

leader_list = ['……','.....','·····']

files = glob.glob("./decrypted_PDFs/*")
for file in files:
    print(os.path.basename(file))
    labels_buffer, sentences_buffer = generate_dataset(file)
    labels += labels_buffer
    sentences += sentences_buffer

ff_fr_20213q4_allj.pdf
yes ·····
Either Index 1 or Index 2 was not detected.
Only Index 2 was detected. We'll extract first page to Index 2 of PDF file.
ページレンジ：　 3  to  7
総行数：　 89
qr2020_q4_f_jp.pdf
yes ……
Both Index1 and 2 were detected.
ページレンジ：　 3  to  19
総行数：　 263
results-fy2021-q1-all.pdf
yes ……
Both Index1 and 2 were detected.
ページレンジ：　 3  to  7
総行数：　 59
BANDAI_NAMCO_202103.pdf
yes ……
Either Index 1 or Index 2 was not detected.
Index was not properly detected from Mokuji page. We'll extract p3-20 from PDF file.
ページレンジ：　 3  to  19
総行数：　 116
210506.pdf
yes ……
Both Index1 and 2 were detected.
ページレンジ：　 3  to  5
総行数：　 43
Olympus_FY2021_Consolidated_Financial_Results_J.pdf
yes ……
Either Index 1 or Index 2 was not detected.
Only Index 2 was detected. We'll extract first page to Index 2 of PDF file.
ページレンジ：　 3  to  18
総行数：　 223
kes202104.pdf
yes ……
Both Index1 and 2 were detected.
ページレンジ：　 3  to  11
総行数：　 69
20210506_cons.pdf
yes ……
Either Index 1 or Index 2 was not detected.
Index was not

In [8]:

print(len(labels))
print(len(sentences))

3188
3188


In [9]:
# pickleに書き出し
pd.to_pickle(labels,"labels_1014_th841.pkl")
pd.to_pickle(sentences,"sentences_1014_th841.pkl")

In [8]:
#Threshold=0.87

print(len(labels))
print(len(sentences))

3232
3232


In [136]:
# pickleデータ読み込み
fuga = pd.read_pickle("sentences.pkl")
fuga

['1.経営成績・財政状態に関する分析 \u3000(1) 経営成績に関する分析\u3000 国内売上高 \u3000 海外売上高 \u3000 売上高 \u3000 営業利益 \u3000 税金等調整前当期純利益 \u3000 当社株主帰属当期純利益 \u3000 為替レート（円／米＄）  為替レート（円／Euro） \u3000（単位：億円） \u30002021 年 3 月期 \u30002020 年 3 月期  増減額  増減率 \u300042.3% \u300057.7% \u3000100.0% \u30007.5% \u300010.8% \u30008.3% \u30009,279  12,646 21,925 \u30001,655 \u30002,359 \u30001,812 \u3000106 円 124 円 \u300043.4% \u300056.6% \u300010,040  △761 △7.6%13,111   △465 △3.5%\u3000100.0% \u300023,151  △1,226 △5.3%\u30001,866  △211 △11.3%\u30008.1% \u30007.5% \u30005.4% \u30001,731 \u30001,250 \u3000109 円 121 円 \u300036.3% \u300045.0% \u3000628 \u3000562 \u3000△3 円3 円\u3000当社グループの 2021 年 3 月期における連結売上高は、バイオ CDMO 事業、医薬品事業、電子材料事業などで売上を伸ばしましたが、フォトイメージング事業、光学・電子映像事業、ドキュメント事業の売上減少などにより 2 兆 1,925 億円（前年同期比 5.3％減）となりました。',
 '   営業利益は、1,655 億円（前年同期比 11.3％減）となりました。',
 '税金等調整前当期純利益は 2,359億円（前年同期比 36.3％増）、当社株主帰属当期純利益は 1,812 億円（前年同期比 45.0％増）となりました。',
 '2021 年 3 月期の円為替レートは、対米ドルで 106 円、対ユーロで 124 円となりました。',
 ' \u3000【事業セグメント別の売上高】\u3000（単位：億円

In [41]:
sample = """株式会社エヌ・ティ・ティ・データ(9613) 2021年3月期 決算短信
住友金属鉱山(株)(5713)2021年3月期決算短信
富士フイルムホールディングス(株)2021年3月期決算短信
ヤマハ株式会社(7951) 2021年3月期 決算短信
オリックス株式会社(8591)2021年3月期 決算短信
武田薬品工業(株)(4502) 2021年3月期 決算短信
セコム株式会社(9735) 2021年3月期 決算短信
花王(株) (4452) 2021年12月期 第1四半期決算短信
(株)バンダイナムコホールディングス(7832)2021年3月期決算短信
任天堂株式会社(7974) 2021年3月期 決算短信
協和キリン(株)(4151)2021年12月期 第1四半期決算短信
オリンパス(株) (7733) 2021年3月期 決算短信
(株)資生堂(4911) 2021年12月期 第1四半期決算短信
三菱地所(株)(8802)2021年3月期 決算短信
栗田工業株式会社(6370) 2021年3月期決算短信
ダイキン工業株式会社(6367) 2021年3月期 決算短信
スズキ株式会社(7269) 2021年3月期 決算短信
日本航空株式会社(9201)2021年3月期 決算短信
AGC(株) (5201) 2021年12月期 第1四半期決算短信
ミネベアミツミ(株) (6479) 2021年3月期 決算短信
パナソニック(株) (6752) 2021年3月期 決算短信
味の素(株)(2802) 2021年3月期決算短信
日本電信電話株式会社(9432) 2020年度 決算短信
ソフトバンクグループ(株) (9984) 2021年3月期 決算短信
ソフトバンク株式会社(9434) 2021年3月期 決算短信
株式会社ミスミグループ本社(9962) 2021年3月期 決算短信
ライオン(株) (4912) 2021年12月期 第1四半期決算短信
"""

#############